# using Microsoft Graph to access Azure resources

“[Build .NET apps with Microsoft Graph](https://learn.microsoft.com/en-us/graph/tutorials/dotnet?tabs=aad)”

## NuGet packages

In [1]:
#r "nuget: Microsoft.Graph"
#r "nuget: Azure.Identity"

Installed Packages Azure.Identity, 1.9.0 Microsoft.Graph, 5.12.0

In [2]:
#r "nuget: SonghayCore"

Installed Packages SonghayCore, 6.0.4

## load conventional `ProgramMetadata`

In [3]:
using System.IO;
using System.Text.Json;
using Songhay.Models;

var json = File.ReadAllText(Environment.GetEnvironmentVariable("SONGHAY_APP_SETTINGS_PATH"));

JsonDocument.Parse(json).RootElement.TryGetProperty(typeof(ProgramMetadata).Name, out var element)

True

In [4]:
var meta = JsonSerializer.Deserialize<ProgramMetadata>(element);
var restMeta = meta.RestApiMetadataSet["RbacVault2023"];

restMeta.ClaimsSet.Keys

[ app-client-id, app-tenant-id, resource-scope, test-secret-id ]

## use `DeviceCodeCredential` for Azure Key Vault

In [5]:
using Azure.Core;
using Azure.Identity;

var options = new DeviceCodeCredentialOptions
{
    ClientId = restMeta.ClaimsSet["app-client-id"],
    TenantId = restMeta.ClaimsSet["app-tenant-id"],
    DeviceCodeCallback = (info, _) =>
        {
            // Display the device code message to
            // the user. This tells them
            // where to go to sign in and provides the
            // code to use.
            Console.WriteLine(info.Message);
            return Task.FromResult(0);
        }
};

var credential = new DeviceCodeCredential(options);

var scopes = new[] { restMeta.ClaimsSet["resource-scope"] };

var context = new TokenRequestContext(scopes);

var response = await credential.GetTokenAsync(context);

!string.IsNullOrWhiteSpace(response.Token)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AMSNUZPQY to authenticate.


True

In [6]:
using System.Net.Http;
using System.Net.Http.Headers;
using Songhay.Extensions;

var client = new HttpClient();
var uri = restMeta.ToUri("UriPathTemplateForTestSecret", restMeta.ClaimsSet["test-secret-id"]);

client.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue("Bearer", response.Token);

var res = await client.GetAsync(uri);

res.Content.ReadAsStringAsync().GetAwaiter().GetResult()

{"value":"this-is-the-secret","id":"https://songhay-key-vault-2023.vault.azure.net/secrets/test-secret/46e124ec22e34fff8e174886edcabe50","attributes":{"enabled":true,"created":1685168096,"updated":1685168096,"recoveryLevel":"Recoverable+Purgeable","recoverableDays":90},"tags":{}}

In [ ]:
using Microsoft.Graph;

var client = new GraphServiceClient(credential, scopes);

var users = await client.Users.GetAsync(config =>
    {
        config.QueryParameters.Select = new[] { "DisplayName", "CompanyName" };
        config.QueryParameters.Top = 5;
        config.QueryParameters.Orderby = new[] { "DisplayName" };
    });

users

In [ ]:
var me = await client.Me.GetAsync();

me

In [ ]:
#r "nuget: Azure.Storage.Blobs"


In [ ]:
using Azure.Storage.Blobs;
using Azure.Storage.Blobs.Models;

var uri = new Uri("https://fubar.blob.core.windows.net", UriKind.Absolute);
var client = new BlobServiceClient(uri, credential);

var containerClient = client.GetBlobContainerClient("foo");

var blobClient = containerClient.GetBlobClient("kb-1064618963.json");

blobClient.DownloadContent()